In [1]:
import pandas as pd
import numpy as np

## Creating Indicator

In [2]:
df = pd.read_csv('Singapore Resident Data Clean.csv')
df.head()


Planning Area Subzone   Total   0  4   5  9  10  14  15  19  20  24  25  29  \
0    Ang Mo Kio   Total  174770   6790   7660    8290    9320   10310   11170   
1         Bedok   Total  289750  11690  13400   14750   16930   19450   19860   
2        Bishan   Total   90700   3430   4330    4710    5520    6860    6460   
3      Boon Lay   Total      30      0      0       0       0       0       0   
4   Bukit Batok   Total  139270   5510   6890    7970    9240   10070   10460   

    30  34  ...  40  44  45  49  50  54  55  59  60  64  65  69  70  74  \
0  12250.0  ...   13710   13000   14010   13800   12980   11050  6670.0   
1  19270.0  ...   22520   21460   23430   23380   20590   16750  9310.0   
2   5720.0  ...    7070    6800    7540    7700    6360    4860  2730.0   
3      0.0  ...       0       0       0       0       0       0     0.0   
4   9760.0  ...   10980   11310   12190   11770    9370    5990  3060.0   

   75  79  80  84  85 & Over  
0  5140.0  3250.0     2300.0  
1  7330.0  4760.0     4010.0  
2  2140.0  1370.0     1090.0  
3     0.0     0.0        0.0  
4  2230.0  1450.0     1090.0  

[5 rows x 21 columns]

In [3]:
df.columns

Index(['Planning Area', 'Subzone', 'Total', '0  4', '5  9', '10  14', '15  19',
       '20  24', '25  29', '30  34', '35  39', '40  44', '45  49', '50  54',
       '55  59', '60  64', '65  69', '70  74', '75  79', '80  84',
       '85 & Over'],
      dtype='object')

In [4]:
df = df.fillna(0)

In [5]:
for col in df.columns[2:]:
    df[col] = df[col].astype('int64')

In [6]:
df['dependent'] = df['60  64'] + df['65  69'] + df['70  74'] + df['75  79'] + df['80  84'] + df['85 & Over']
df['rescuer'] = df['15  19'] + df['20  24'] + df['25  29'] + df['30  34'] + df['35  39'] + df['40  44'] + df['45  49'] + df['50  54'] + df['55  59']

In [7]:
indicator = []
for index, row in df.iterrows():
    try:
        ind = row['dependent'] / row['rescuer']
        indicator.append(ind)
    except:
        indicator.append(1)
df['indicator'] = indicator

In [9]:
df[['Planning Area', 'indicator']].to_csv('Dependency Indicator.csv')

In [8]:
with open('planning-boundary-area.kml') as f:
    kml = f.read()

In [17]:
import re
coordinates = re.findall("<coordinates>(.*?)</coordinates>", kml)
names = re.findall('<SimpleData name="PLN_AREA_N">(.*?)</SimpleData>', kml)

In [34]:
def process_poly(poly_str):
    longtitudes = []
    latitudes = []
    poly = [i[:-4] for i in poly_str.split(' ')]
    for c in poly:
        long, lat = c.split(',')
        longtitudes.append(long)
        latitudes.append(lat)
    longtitudes = np.array(longtitudes).astype(np.float32)
    latitudes = np.array(latitudes).astype(np.float32)
    return longtitudes, latitudes


In [35]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

def check_inside(long, lat, poly_longs, poly_lats):
    lons_lats_vect = np.column_stack((poly_longs, poly_lats))
    polygon = Polygon(lons_lats_vect)
    point = Point(long, lat)
    return polygon.contains(point)

In [38]:
poly_list = []
for coord in coordinates:
    a, b = process_poly(coord)
    poly_list.append((a,b))

In [49]:
poly_list

[(array([103.817406, 103.81787 , 103.81841 , 103.81841 , 103.81855 ,
         103.81855 , 103.818665, 103.81886 , 103.8191  , 103.82162 ,
         103.82169 , 103.82178 , 103.82178 , 103.82182 , 103.82183 ,
         103.82275 , 103.823166, 103.82332 , 103.82338 , 103.823494,
         103.8237  , 103.8238  , 103.8238  , 103.82391 , 103.82425 ,
         103.82439 , 103.82449 , 103.82465 , 103.82478 , 103.824974,
         103.825066, 103.82521 , 103.82575 , 103.82592 , 103.825935,
         103.8261  , 103.82743 , 103.8286  , 103.829254, 103.82959 ,
         103.82984 , 103.83013 , 103.83048 , 103.83072 , 103.83083 ,
         103.83087 , 103.83092 , 103.83105 , 103.83115 , 103.83128 ,
         103.83163 , 103.8318  , 103.83182 , 103.831825, 103.831985,
         103.83222 , 103.83237 , 103.83251 , 103.83267 , 103.832695,
         103.832855, 103.83302 , 103.833206, 103.83343 , 103.83352 ,
         103.83362 , 103.833664, 103.83371 , 103.83387 , 103.83397 ,
         103.834145, 103.834305, 1

## Creating Dummy Data

In [40]:
add = pd.read_csv('address_coord_postcode.csv')
add.head()


ADDRESS BLK_NO  \
0  1 STRAITS BOULEVARD SINGAPORE CHINESE CULTURAL...      1   
1  11A STRAITS BOULEVARD TEMPORARY SITE OFFICE SI...    11A   
2           5A MARINA GARDENS DRIVE SINGAPORE 018910     5A   
3  2 CENTRAL BOULEVARD CENTRAL BOULEVARD TOWERS S...      2   
4  21 PARK STREET CITIBANK TRADE_BRANCH SINGAPORE...     21   

                            BUILDING  LATITUDE   LONGITUDE  LONGTITUDE  \
0  SINGAPORE CHINESE CULTURAL CENTRE  1.275805  103.849615  103.849615   
1              TEMPORARY SITE OFFICE  1.274950  103.851665  103.851665   
2                                NIL  1.279587  103.868956  103.868956   
3           CENTRAL BOULEVARD TOWERS  1.279744  103.851591  103.851591   
4              CITIBANK TRADE_BRANCH  1.276424  103.854759  103.854759   

   POSTAL             ROAD_NAME                                 SEARCHVAL  \
0   18906     STRAITS BOULEVARD         SINGAPORE CHINESE CULTURAL CENTRE   
1   18907     STRAITS BOULEVARD                     TEMPORARY SITE OFFICE   
2   18910  MARINA GARDENS DRIVE  5A MARINA GARDENS DRIVE SINGAPORE 018910   
3   18916     CENTRAL BOULEVARD                  CENTRAL BOULEVARD TOWERS   
4   18925           PARK STREET                     CITIBANK TRADE_BRANCH   

             X            Y  
0  29813.66349  28697.52076  
1  30041.83890  28602.98724  
2  31966.12079  29115.75337  
3  30033.60446  29133.10468  
4  30386.13730  28766.04895

In [41]:
len(add)

141902

In [42]:
from random import randrange
import datetime 

def random_date(start,l):
   current = start
   while l >= 0:
      curr = current + datetime.timedelta(seconds=randrange(10*24*3600))
      yield curr
      l-=1
    
startDate = datetime.datetime(2020, 6, 14, 00, 00, 00)

In [43]:
# Creating 100 cases over 10 days
import random

index = random.sample(range(0, 141901), 100)
is_diabetes = 30*[1] + 70*[0]
random.shuffle(is_diabetes)
is_heartdisease = 25*[1] + 75*[0]
random.shuffle(is_heartdisease)
is_stroke = 25*[1] + 75*[0]
random.shuffle(is_stroke)
blood_type = 25*['A', 'B', 'AB', 'O']
random.shuffle(blood_type)
age = np.random.normal(70, 5, 100).astype(np.uint32)
time = [x.strftime("%d/%m/%y %H:%M:%S") for x in random_date(startDate,99)]


In [44]:
len(time)

100

In [45]:
dummy = add.iloc[index]
dummy['is_diabetes'] = is_diabetes
dummy['is_heartdisease'] = is_heartdisease
dummy['is_stroke'] = is_stroke
dummy['blood_type'] = blood_type
dummy['age'] = age
dummy['time']= time

c:\users\maula\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\maula\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\maula\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be

In [46]:
dummy = dummy.sort_values('time')
dummy.reset_index(inplace = True)
dummy.drop('index', axis = 1, inplace = True)
dummy.head()

ADDRESS BLK_NO  \
0      12 TUAS LOOP TUAS CONNECTION SINGAPORE 637346     12   
1      11 JALAN RUMBIA THE IMPERIAL SINGAPORE 239621     11   
2  132 BUKIT BATOK WEST AVENUE 6 OCBC Bukit Batok...    132   
3                     6A ROBIN LANE SINGAPORE 258236     6A   
4  22 JALAN SUKACHITA SERANGOON PARK SINGAPORE 35...     22   

                                 BUILDING  LATITUDE   LONGITUDE  LONGTITUDE  \
0                         TUAS CONNECTION  1.318824  103.635486  103.635486   
1                            THE IMPERIAL  1.294745  103.842729  103.842729   
2  OCBC Bukit Batok West Ave 6 - 7 Eleven  1.350419  103.745717  103.745717   
3                                     NIL  1.319373  103.826734  103.826734   
4                          SERANGOON PARK  1.347769  103.866584  103.866584   

   POSTAL                  ROAD_NAME                               SEARCHVAL  \
0  637346                  TUAS LOOP                         TUAS CONNECTION   
1  239621               JALAN RUMBIA                            THE IMPERIAL   
2  650132  BUKIT BATOK WEST AVENUE 6  OCBC Bukit Batok West Ave 6 - 7 Eleven   
3  258236                 ROBIN LANE          6A ROBIN LANE SINGAPORE 258236   
4  358898            JALAN SUKACHITA                          SERANGOON PARK   

              X            Y  is_diabetes  is_heartdisease  is_stroke  \
0   5983.081936  33455.29048            0                1          0   
1  29047.280840  30791.89092            0                0          0   
2  18250.890390  36948.20630            0                0          1   
3  27267.148400  33515.12911            0                1          0   
4  31702.077880  36654.99553            0                0          0   

  blood_type  age               time  
0          B   69  14/06/20 01:51:08  
1          O   68  14/06/20 03:26:45  
2          B   67  14/06/20 08:37:18  
3         AB   68  14/06/20 13:41:15  
4          O   73  14/06/20 16:21:36

In [47]:
dummy.to_csv('Dummy Data.csv')

In [52]:
areas = []
for i, c in dummy.iterrows():
    lat = c['LATITUDE']
    long = c['LONGTITUDE']
    for ind, poly in enumerate(poly_list):
        poly_longs, poly_lats = poly
        flag = check_inside(long, lat, poly_longs, poly_lats)
        if flag == True:
            break
    areas.append(names[ind].lower())

In [54]:
dummy['area'] = areas

In [72]:
df['area'] = df['Planning Area'].apply(lambda x: x.lower())

In [73]:
final = pd.merge(dummy, df, how = 'left', on='area')

In [78]:
final[['ADDRESS', 'BLK_NO', 'BUILDING', 'LATITUDE', 'LONGTITUDE', 'ROAD_NAME', 'indicator', 'area', 'age', 'is_diabetes', 'is_heartdisease', 'is_stroke']].head()

ADDRESS BLK_NO  \
0      12 TUAS LOOP TUAS CONNECTION SINGAPORE 637346     12   
1      11 JALAN RUMBIA THE IMPERIAL SINGAPORE 239621     11   
2  132 BUKIT BATOK WEST AVENUE 6 OCBC Bukit Batok...    132   
3                     6A ROBIN LANE SINGAPORE 258236     6A   
4  22 JALAN SUKACHITA SERANGOON PARK SINGAPORE 35...     22   

                                 BUILDING  LATITUDE  LONGTITUDE  \
0                         TUAS CONNECTION  1.318824  103.635486   
1                            THE IMPERIAL  1.294745  103.842729   
2  OCBC Bukit Batok West Ave 6 - 7 Eleven  1.350419  103.745717   
3                                     NIL  1.319373  103.826734   
4                          SERANGOON PARK  1.347769  103.866584   

                   ROAD_NAME  indicator         area  age  is_diabetes  \
0                  TUAS LOOP   1.000000         tuas   69            0   
1               JALAN RUMBIA   0.214286       museum   68            0   
2  BUKIT BATOK WEST AVENUE 6   0.242269  bukit batok   67            0   
3                 ROBIN LANE   0.289712      tanglin   68            0   
4            JALAN SUKACHITA   0.332279    serangoon   73            0   

   is_heartdisease  is_stroke  
0                1          0  
1                0          0  
2                0          1  
3                1          0  
4                0          0

In [79]:
final[['ADDRESS', 'BLK_NO', 'BUILDING', 'LATITUDE', 'LONGTITUDE', 'ROAD_NAME', 'indicator', 'area', 'age', 'is_diabetes', 'is_heartdisease', 'is_stroke']].to_csv('Final Simulation Data.csv')